## 17. 바이트 페어 인코딩(Byte Pair Encoding, BPE)
- 단어를 단어 집합에 없는 단어란 의미에서 OOV(Out-Of-Vocabulary) 또는 UNK(Unknown Token)라고 표현
- 모르는 단어로 인해 문제를 푸는 것이 까다로워지는 상황을 OOV 문제
- 데이터 압축 알고리즘
- BPE은 기본적으로 연속적으로 가장 많이 등장한 글자의 쌍을 찾아서 하나의 글자로 병합하는 방식을 수행
- BPE을 요약하면, 글자(charcter) 단위에서 점차적으로 단어 집합(vocabulary)을 만들어 내는 Bottom up 방식의 접근을 사용
- 훈련 데이터에 있는 단어들을 모든 글자(chracters) 또는 유니코드(unicode) 단위로 단어 집합(vocabulary)를 만들고, 가장 많이 등장하는 유니그램을 하나의 유니그램으로 통합

- ex) aaabdaaabac (ZabdZabac)
    - 1) 가장 자주 등장하고 있는 바이트의 쌍(byte pair)은 'aa', 'aa'라는 바이트의 쌍을 'Z'로 치환
    - 2) 문자열 중에서 가장 많이 등장하고 있는 바이트의 쌍은 'ab', 'ab'를 'Y'로 치환
    - 3) 가장 많이 등장하고 있는 바이트의 쌍은 'ZY', 이를 'X'로 치환

- ex2) lower, lower, newest, widest
    - dictionary [l o w : 5,  l o w e r : 2,  n e w e s t : 6,  w i d e s t : 3]
    - vocabulary [l, o, w, e, r, n, w, s, t, i, d]
    - (1) dictionary update (e, s)의 쌍을 es로 통합 
        - update된 dictionary : l o w : 5, / l o w e r : 2, / n e w es t : 6, / w i d es t : 3
        - update된 vocabulary : l, o, w, e, r, n, w, s, t, i, d, es
    - (2) 빈도수가 9로 가장 높은 (es, t)의 쌍을 est로 통합
        - update된 dictionary : l o w : 5, / l o w e r : 2, / n e w est : 6, / w i d est : 3
        - update된 vocabulary : l, o, w, e, r, n, w, s, t, i, d, es, est

    - (3) 빈도수가 7로 가장 높은 (l, o)의 쌍을 lo로 통합
        - update된 dictionary : lo w : 5, / lo w e r : 2, / n e w est : 6, / w i d est : 3
        - update된 vocabulary : l, o, w, e, r, n, w, s, t, i, d, es, est, lo
    - (4) 이와 같은 방식으로 총 n회 반복
        - update된 dictionary : low : 5, / low e r : 2, / newest : 6, / widest : 3
        - update된 vocabulary : l, o, w, e, r, n, w, s, t, i, d, es, est, lo, low, ne, new, newest, wi, wid, widest

![using_bpe](img/using_bpe.png)

- 'lowest'를 기계는 'low'와 'est' 두 단어로 인코딩합니다. 그리고 이 두 단어는 둘 다 단어 집합에 있는 단어이므로 OOV가 아님
- OOV 단어가 나오면 loki => ('lo', 'k', 'i')로 분리 lo는 유지하고 'k'dhk 'i'는 분리




In [2]:
import re, collections
from IPython.display import display, Markdown, Latex

# BPE 수행 횟수
num_merges = 10
dictionary = {'l o w </w>' : 5,
         'l o w e r </w>' : 2,
         'n e w e s t </w>':6,
         'w i d e s t </w>':3
         }

In [3]:
# BPE의 코드. 
# 알고리즘은 위에서 설명했던 것과 동일하게 가장 빈도수가 높은 유니그램의 쌍을 하나의 유니그램으로 통합하는 과정으로 num_merges회 반복

def get_stats(dictionary):
    # 유니그램의 pair들의 빈도수를 카운트
    pairs = collections.defaultdict(int)
    for word, freq in dictionary.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    print('현재 pair들의 빈도수 :', dict(pairs))
    return pairs

def merge_dictionary(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out


In [4]:
bpe_codes = {}
bpe_codes_reverse = {}

for i in range(num_merges):
    display(Markdown("### Iteration {}".format(i + 1)))
    pairs = get_stats(dictionary)
    best = max(pairs, key=pairs.get)
    dictionary = merge_dictionary(best, dictionary)

    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best

    print("new merge: {}".format(best))
    print("dictionary: {}".format(dictionary))

### Iteration 1

현재 pair들의 빈도수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 8, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('e', 's'): 9, ('s', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'e'): 3}
new merge: ('e', 's')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


### Iteration 2

현재 pair들의 빈도수 : {('l', 'o'): 7, ('o', 'w'): 7, ('w', '</w>'): 5, ('w', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('n', 'e'): 6, ('e', 'w'): 6, ('w', 'es'): 6, ('es', 't'): 9, ('t', '</w>'): 9, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'es'): 3}
new merge: ('es', 't')
dictionary: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


### Iteration 3

...

현재 pair들의 빈도수 : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('new', 'est</w>'): 6, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('new', 'est</w>')
dictionary: {'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 9

현재 pair들의 빈도수 : {('low', '</w>'): 5, ('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('low', '</w>')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 10

현재 pair들의 빈도수 : {('low', 'e'): 2, ('e', 'r'): 2, ('r', '</w>'): 2, ('w', 'i'): 3, ('i', 'd'): 3, ('d', 'est</w>'): 3}
new merge: ('w', 'i')
dictionary: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}
